# **Übung 5: Rollgeschwindigkeit bei Variation der Streckung**

In [ ]:
# Importieren von Python-Biblioteken
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, IntSlider, FloatSlider, fixed, VBox, HBox, Label
from mpl_toolkits.mplot3d import axes3d, axis3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import ipywidgets as widgets

# Erhöhen der Plot-Auflösung
plt.rcParams["figure.dpi"] = 140

### 4.1.1 Visualisierung: Rechteckflügel mit Querrudern

In [ ]:
def rechteckfluegel(AR, xi, l_i, l_a):
    """Generiert die Planform eines Rechteckflügels mit Rudern"""
    
    b = l_a * AR
    s = b / 2
    l_i = l_a

    # ruderbreite und -tiefe 
    y_ruder = 0.65 * s
    x_ruder = 0.8 * l_i
    rudertiefe = l_i - x_ruder
    
    # Flügelpunkte
    points_fluegel = np.array([
        [0, 0], [s, 0], [s, x_ruder], [y_ruder, x_ruder], 
        [y_ruder, l_a], [-y_ruder, l_a], [-y_ruder, x_ruder], [-s, x_ruder], [-s, 0], [0, 0]
    ])
    points_fluegel = [np.append(pt, 0) for pt in points_fluegel]
    
    # Ruderpunkte links
    ruder_links = np.array([
        [y_ruder, x_ruder, 0], [s, x_ruder, 0], 
        [s, x_ruder + rudertiefe * np.cos(np.radians(xi)), -rudertiefe * np.sin(np.radians(xi))], 
        [y_ruder, x_ruder + rudertiefe * np.cos(np.radians(xi)), -rudertiefe * np.sin(np.radians(xi))],
        [y_ruder, x_ruder, 0]
    ])
    
    # Ruderpunkte rechts
    ruder_rechts = np.array([
        [-y_ruder, x_ruder, 0], [-s, x_ruder, 0], 
        [-s, x_ruder + rudertiefe * np.cos(np.radians(xi)), -rudertiefe * np.sin(np.radians(xi))], 
        [-y_ruder, x_ruder + rudertiefe * np.cos(np.radians(xi)), -rudertiefe * np.sin(np.radians(xi))],
        [-y_ruder, x_ruder, 0]
    ])
    
    return points_fluegel, ruder_links, ruder_rechts

def plot_rechteckfluegel(AR, xi, l_i=1.0, l_a=1.0):
    """Plottet die Planform eines Rechteckflügels mit verstellbaren Rudern"""
    
    points_fluegel, ruder_links, ruder_rechts = rechteckfluegel(AR, xi, l_i, l_a)

    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Flügel plotten
    fluegel = Poly3DCollection([points_fluegel], color="black", alpha=0.6)
    ax.add_collection3d(fluegel)
    
    # Ruder links plotten
    ruder_l = Poly3DCollection([ruder_links], color="lightcoral", alpha=0.6)
    ax.add_collection3d(ruder_l)
    
    # Ruder rechts plotten
    ruder_r = Poly3DCollection([ruder_rechts], color="lightcoral", alpha=0.6)
    ax.add_collection3d(ruder_r)
    
    # Achsen skalieren und Label setzen
    ax.grid(False)

    ax.set_yticks([])
    ax.set_zticks([])

    ax.view_init(elev=5, azim=30)
    plt.gca().set_aspect('equal')
    plt.show()

def widget_rechteckfluegel():
    """Erzeugt ein Widget, mit dem sich die Planform eines Rechteckflügels mit verstellbaren Rudern visualisieren lässt"""

    slider_AR = VBox([Label('Streckung'), FloatSlider(value=10, min=5, max=20, step=1)])
    slider_xi = VBox([Label('Klappenwinkel [°]'), FloatSlider(value=15, min=-15, max=40, step=1)])
    
    # Interaktiver plot
    ui = VBox([slider_AR,  slider_xi])
    out = widgets.interactive_output(plot_rechteckfluegel, {'AR': slider_AR.children[1], 'xi': slider_xi.children[1]
    })

    # Layout anpassen
    out.layout = widgets.Layout(flex='3 1 auto', width='70%')
    ui.layout = widgets.Layout(flex='1 1 auto', width='30%')
    return HBox([out, ui], layout=widgets.Layout(justify_content='center', align_items='center', width='100%'))

In [ ]:
widget_rechteckfluegel()

### 4.1.2 Geometrische Parameter und Randbedinungen

In [ ]:
# Konstant
alpha = 0                       # Anstellwinel [deg]
alpha = alpha / 180 * np.pi     # Anstellwinel [rad]  
U_inf = 10.0                    # Anströmgeschwindigkeit [m/s]
rho = 1.225                     # Dichte [kg/m^3]
l_i = 1.0                       # Flügeltiefe Rumpfmitte [m]
l_a = 1.0                       # Flügeltiefe außen [m]
l_r = 0.2                       # Rudertiefe [m]

#### 🟠

In [ ]:
# Diese Parameter können verändert werden
AR = 10                         # Streckung/ Aspect Ratio
xi = 15                         # Klappenwinkel [deg]
omega = 0                       # dimensionslose Rollgeschwindigkeit

#### Berechnung weiterer geometrischer Parameter

In [ ]:
# Berechnung der Halbspannweite
b = (l_i + l_a) * AR / 2
s = b / 2 

# Berechnung der Flügelfläche aus Halbspannweite und Flügeltiefen
S = b/2 * (l_a + l_i)       

# Verhältnis Rudertiefe-Flügeltiefe und spannweitige Position
lamda = l_r / l_i
y_ruder = 0.65 * s

# Berechnung des Nullanstellwinkels des Klappenprofils
xi = xi * np.pi / 180
alpha_0 = - 2 / np.pi * (np.asin(np.sqrt(lamda)) + np.sqrt(lamda * (1 - lamda))) * xi

print("Halbspannweite:                          ", "%.2f" % s, "m")
print("Spannweitige Position der Ruder:         ", "%.2f" % y_ruder, "m")
print("Nullanstellwinkel des Klappenprofils:    ", str(round(alpha_0, 2)) + "°")